In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dask.distributed import Client, LocalCluster

#client = Client() # Note that `memory_limit` is the limit **per worker**.
client = Client(n_workers=1,
                threads_per_worker=4,
                memory_limit='16GB')
client # If you click the dashboard link in the output, you can monitor real-time progress and get other cool visualizations.

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58064,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:58069,Total threads: 4
Dashboard: http://127.0.0.1:58070/status,Memory: 14.90 GiB
Nanny: tcp://127.0.0.1:58067,


In [3]:
import os
import copy
import sys
import xarray as xr
import numpy as np
import dask.array as da

import matplotlib.pyplot as plt
import hvplot.xarray
import holoviews as hv
import scipy.constants
import scipy.signal as sp

sys.path.append("..")
import processing_dask as pr
import plot_dask
import processing as old_processing

sys.path.append("../../preprocessing/")
from generate_chirp import generate_chirp

/Users/abroome/miniconda3/envs/sprinkles/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# file path to data and configs

prefixes = []

#prefix = "/home/radioglaciology/thomas/radar_data/20230721-summit-day1-bench/20230721_104552"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_103641"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_103845" # 6 db higher tx
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104059"
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104248" # another 6 db
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104456" # 3db higher rx
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_104742" # 20 -> 30 mhz chirp bw
#prefix = "/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105025" # bw back to 20, 15 mhz lo offset
# prefixes.append("/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105509") # 100k pulses, 30 us transmit, 90 us rx
# prefixes.append("/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_105921") # 100k pulses, 10 us transmit, 90 us rx
# prefixes.append("/home/radioglaciology/thomas/radar_data/20230723-summit-day3-2start/20230723_110253") # 100k pulses, 5 us transmit, 60 us rx

# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_145418")
# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_150425")
# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_150853")

# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_151432") # 20 us transmit
# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_153254") # 10 us transmit
# prefixes.append("/media/radioglaciology/Extreme SSD/summit_2023_mapper/20230723_152404") #  5 us transmit

# prefixes.append("/Volumes/Extreme SSD/Summit2023/07232023_anna/20230723_152404")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07232023_anna/20230723_153254")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07232023_anna/20230723_151432")

# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_133510")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_135544")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_143151")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_141923")

#prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_133510")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_134737")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_141923")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_152843")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_155902")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_161051")
#prefixes.append("/Volumes/Extreme SSD/Summit2023/07252023_anna/20230725_162504")

# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_105752")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_113446")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_125920")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_133931")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_152434")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_153206")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_154208")

# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_141235")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_145823")

# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_154544")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_154744")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_154903")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_154955")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_155042")
# prefixes.append("/Volumes/Extreme SSD/Summit2023/08022023_anna/20230802_155153")

#prefixes.append("/Volumes/T7 Shield/Summit2023/08042023_anna/20230804_154502")
#prefixes.append("/Volumes/T7 Shield/Summit2023/08042023_anna/20230804_155432")
prefixes.append("/Volumes/T7 Shield/Summit2023/08042023_anna/20230804_161102")
#prefixes.append("/Volumes/T7 Shield/Summit2023/08042023_anna/20230804_164805")

raw_datasets = []
for prefix in prefixes:
    # resave data as zarr for dask processing
    zarr_path = pr.save_radar_data_to_zarr(prefix)

    # open zarr file, adjust chunk size to be 10 MB - 1 GB based on sample rate/bit depth
    raw = xr.open_zarr(zarr_path, chunks={"pulse_idx": 1000})
    
    raw = pr.invert_phase_dithering(raw, phase_codes_filename='/Users/abroome/Documents/SDR/uhd_radar/phases_1M.bin')
    raw = pr.remove_errors(raw)
    
    raw_datasets.append(raw)

/Users/abroome/miniconda3/envs/sprinkles/lib/python3.9/site-packages/xarray/core/dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "pulse_idx" starting at index 1000. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [5]:
#zero_sample_idx = 36 # X310, fs = 20 MHz
zero_sample_idx = 63 # X310, fs = 50 MHz
#zero_sample_idx = 159 # B205mini, fs = 56 MHz

modify_rx_window = False # set to true if you want to window the reference chirp only on receive, false uses ref chirp as transmitted in config file
rx_window = "blackman" # what you want to change the rx window to if modify_rx_window is true

dielectric_constant = 3.17 # ice (air = 1, 66% velocity coax = 2.2957)
sig_speed = scipy.constants.c / np.sqrt(dielectric_constant)

n_stack = 99800

In [6]:
plots = []

for raw in raw_datasets:
    # Generate reference chirp
    if modify_rx_window:
        config = copy.deepcopy(raw.config)
        config['GENERATE']['window'] = rx_window
    else:
        config = raw.config
        
    config = copy.deepcopy(raw.config)
    config['GENERATE']['lo_offset_sw'] = 12.5e6

    chirp_ts, ref_chirp = generate_chirp(config)
    
    if raw.config['GENERATE']['sample_rate'] == 20e6:
        zero_sample_idx = 36
    elif raw.config['GENERATE']['sample_rate'] == 50e6:
        zero_sample_idx = 63
    elif raw.config['GENERATE']['sample_rate'] == 100e6:
        zero_sample_idx = 81
    else:
        print("different bandwidth, need to specify zero sample index")
    
    # detrend
    # detrended = xr.apply_ufunc(
    #     lambda x: sp.detrend(x, axis=0, type='constant'),
    #     raw,
    #     dask="parallelized"
    # )
    # stacked = pr.stack(detrended, n_stack)

    # Stack and pulse compress
    stacked = pr.stack(raw, 300000) #len(raw.pulse_idx))
    compressed = pr.pulse_compress(stacked, ref_chirp,
                                fs=raw.config['GENERATE']['sample_rate'],
                                zero_sample_idx=zero_sample_idx,
                                signal_speed=sig_speed)

    compressed_power = xr.apply_ufunc(
        lambda x: 20*np.log10(np.abs(x)),
        compressed,
        dask="parallelized"
    )

    plots.append(compressed_power.radar_data[0,:].hvplot.line(label=f"{raw.basename}: n = {len(raw.pulse_idx)}"))

/Users/abroome/Documents/SDR/uhd_radar/postprocessing/notebooks/../processing_dask.py:285: PerformanceWarning: Reshaping is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array.reshape(shape)

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array.reshape(shape)Explictly passing ``limit`` to ``reshape`` will also silence this warning
    >>> array.reshape(shape, limit='128 MiB')
  return data.coarsen({'pulse_idx': n_stack},
/Users/abroome/miniconda3/envs/sprinkles/lib/python3.9/site-packages/dask/array/utils.py:103: ComplexWarning: Casting complex values to real discards the imaginary part
  meta = meta.astype(dtype)
/Users/abroome/miniconda3/envs/sprinkles/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is requir

type: Attempted to run task ('lambda-lambda_0-mean_agg-aggregate-mean_chunk-rechunk-merge-reshape-transpose-vectorize_lambda-vectorize_lambda_0-g--550', 0) on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:58114. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

In [ ]:
plot1D = hv.Overlay(plots)
plot1D = plot1D.opts(xlabel='Reflection Distance (m)', ylabel='Return Power (dB)', height=600)
#plot1D = plot1D.opts(xlim=(0, 500), ylim=(-80, 0), show_grid=True)
plot1D

In [ ]:
#hvplot.save(plot1D, f"/Users/abroome/Desktop/Summit2023/figs/20230725_pulse_lengths_hammingrx.png")
hvplot.save(plot1D, f"/Users/abroome/Desktop/Summit2023/figs/20230802_n100.html")

In [ ]:
import scipy.io

data_dict = {"pc": compressed.radar_data, "dist": compressed.reflection_distance}

scipy.io.savemat(f"outputs/{raw.basename}_pc.mat", mdict=data_dict)

In [ ]:
len(raw.pulse_idx)

In [ ]:
plots = []

for raw in raw_datasets:
    plots.append(np.real(raw.radar_data[:,0]).hvplot.line(x='fast_time', label=f'Real:{raw.basename}') * np.imag(raw.radar_data[:,0]).hvplot.line(x='fast_time', label=f'Imag:{raw.basename}'))



In [ ]:
amp_plot = hv.Overlay(plots[0:3])
amp_plot.opts(xlabel='Fast Time (s)', ylabel='Raw Amplitude')
amp_plot